In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

In [2]:
# potential keys that match up to property loss
Off_to_PL_codes = {
    '200':'Burned', #'Arson',
    '220':'Stolen', #'Burglary',
    '250':'Counterfeited/Forged', #'Counterfeiting/Forgery',
    '290':'Destroyed/Damaged/Vandalized', # 'Destruction/Damage/Vandalism',
    # 23-series likely
    '23A':'Stolen',
    '23B':'Stolen',
    '23C':'Stolen',
    '23D':'Stolen',
    '23E':'Stolen',
    '23F':'Stolen',
    '23G':'Stolen',
    '23H':'Stolen',
    '240':'Stolen', #'Motor Vehicle Theft'
}

# Checking compiled PL file for unique PL codes, yield only 4: 
# 'Stolen', 'Destroyed/Damaged/Vandalized', 'Counterfeited/Forged', 'Burned'

In [3]:
pl_df = pd.read_csv('../Data/Crime & COVID (2018-2020)/Compiled/2018-2020_Compiled_Property_Data.csv', encoding='ISO-8859-1')
pl_df.drop('Unnamed: 0', axis=1, inplace=True)
pl_df['ID_Key'] = pl_df['ORI'] + '<>' + pl_df['Incident Number'] + '<>' + pl_df['Incident Date']
pl_df.head()

,ID_Key,ORI,Incident Number,Incident Date,Report Date Indicator,Type of Property Loss,Property Description,Property Value,Agency Name
0,TN0020100<>2208260022<>01/01/2018,TN0020100,2208260022,01/01/2018,NaN,Stolen,Identity-Intangible,0.0,Shelbyville Police Department
1,TNMPD0000<>1801000398ME<>01/01/2018,TNMPD0000,1801000398ME,01/01/2018,NaN,Stolen,Other,40.0,Memphis Police Department
2,TN0950100<>2018000104<>01/01/2018,TN0950100,2018000104,01/01/2018,NaN,Stolen,Firearms,450.0,Lebanon Police Department
3,TN0950100<>2018000104<>01/01/2018,TN0950100,2018000104,01/01/2018,NaN,Stolen,Clothes/Furs,200.0,Lebanon Police Department
4,TN0950100<>2018000016<>01/01/2018,TN0950100,2018000016,01/01/2018,NaN,Stolen,Alcohol,27.0,Lebanon Police Department


In [8]:
pl_df['Type of Property Loss'].value_counts()

Stolen                          956573
Destroyed/Damaged/Vandalized    157184
Counterfeited/Forged             18014
Burned                            2779
Name: Type of Property Loss, dtype: int64

In [4]:
df = pd.read_csv('../Data/Crime & COVID (2018-2020)/Compiled/2018-2020_Compiled_Offense_Data.csv', encoding='ISO-8859-1')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.ID_Key = df['ORI'] + '<>' + df['Incident Number'] + '<>' + df['Incident Date']
df.head()

,ID_Key,ORI,Incident Number,Incident Date,Report Date Indicator,Offense Code,Offense Type,Attempted or Completed,Number of Offenses in Incident,Number of Victims in Incident,...,Number of Premises Entered,Method of Entry,Primary Location Type,Secondary Location Type,Identity Theft,Home Invasion,Drug Type,Drug Origin,Drug Related,Drug Precursor
0,TN0020100<>2208260022<>01/01/2018,TN0020100,2208260022,01/01/2018,NaN,26C,Fraud - Impersonation,Completed,2,1,...,NaN,Missing,Residence/Home,NaN,Identity Theft,Missing,Missing,Missing,Not Drug Related,Missing
1,TN0330700<>18-00856<>01/01/2018,TN0330700,18-00856,01/01/2018,NaN,270,Embezzlement,Completed,1,1,...,NaN,Missing,"Specialty store (TV, Fur, etc.)",NaN,No Identity Theft,Missing,Missing,Missing,Not Drug Related,Missing
2,TN0470000<>1801010036<>01/01/2018,TN0470000,1801010036,01/01/2018,NaN,35A,Drug/Narcotic Violations,Completed,1,1,...,NaN,Missing,Highway/Road/Alley,NaN,Missing,Missing,Cultivated Marijuana,Outdoor Marijuana Growth,Drug Related,Missing
3,TN0470000<>1801010038<>01/01/2018,TN0470000,1801010038,01/01/2018,NaN,220,Burglary,Completed,1,1,...,NaN,Forced Entry,Residence/Home,NaN,No Identity Theft,Missing,Missing,Missing,Not Drug Related,Missing
4,TN0470000<>1801010039<>01/01/2018,TN0470000,1801010039,01/01/2018,NaN,13A,Aggravated Assault,Completed,1,1,...,NaN,Missing,Residence/Home,NaN,Missing,Missing,Missing,Missing,Not Drug Related,Missing


In [6]:
# iterate through ID_Keys to find those offenses that include a property loss-related offense
# sub_df = df[df['Offense Code']]
df.loc[:,'PL_Detected'] = 0

for id in df.ID_Key.unique():
    # get slice
    off_subset = df.loc[df.ID_Key == id]
    
    # is there a property loss relevant charge?
    for code in list(off_subset['Offense Code'].values):
        if code in list(Off_to_PL_codes.keys()):
            print(f'Property loss detected, code: {code}.')
            # get index
            row_ind = off_subset.loc[off_subset['Offense Code'] == code].index[0]
            # flag df for PL
            df.iloc[row_ind]['PL_Detected'] = 1
    # if different hours of day, then treat as separate crime events

    break

Property loss detected, code: 23H.


/var/folders/pt/y3bb3b4j7yz3zzczhxc5l9p0vbrddc/T/ipykernel_9981/140190545.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[row_ind]['PL_Detected'] = 1


In [6]:
df.head()

,ID_Key,ORI,Incident Number,Incident Date,Report Date Indicator,Offense Code,Offense Type,Attempted or Completed,Number of Offenses in Incident,Number of Victims in Incident,...,Method of Entry,Primary Location Type,Secondary Location Type,Identity Theft,Home Invasion,Drug Type,Drug Origin,Drug Related,Drug Precursor,PL_Detected
0,TN0020100<>2208260022<>01/01/2018,TN0020100,2208260022,01/01/2018,NaN,26C,Fraud - Impersonation,Completed,2,1,...,Missing,Residence/Home,NaN,Identity Theft,Missing,Missing,Missing,Not Drug Related,Missing,0
1,TN0330700<>18-00856<>01/01/2018,TN0330700,18-00856,01/01/2018,NaN,270,Embezzlement,Completed,1,1,...,Missing,"Specialty store (TV, Fur, etc.)",NaN,No Identity Theft,Missing,Missing,Missing,Not Drug Related,Missing,0
2,TN0470000<>1801010036<>01/01/2018,TN0470000,1801010036,01/01/2018,NaN,35A,Drug/Narcotic Violations,Completed,1,1,...,Missing,Highway/Road/Alley,NaN,Missing,Missing,Cultivated Marijuana,Outdoor Marijuana Growth,Drug Related,Missing,0
3,TN0470000<>1801010038<>01/01/2018,TN0470000,1801010038,01/01/2018,NaN,220,Burglary,Completed,1,1,...,Forced Entry,Residence/Home,NaN,No Identity Theft,Missing,Missing,Missing,Not Drug Related,Missing,0
4,TN0470000<>1801010039<>01/01/2018,TN0470000,1801010039,01/01/2018,NaN,13A,Aggravated Assault,Completed,1,1,...,Missing,Residence/Home,NaN,Missing,Missing,Missing,Missing,Not Drug Related,Missing,0


In [25]:
off_subset.loc[off_subset['Offense Code'] == code].index[0]

243

In [ ]:
for i in range(len(pl_df)):
    X = pl_df.iloc[i]
    id = X.ID_Key
    Y = df.loc[df.ID_Key == id]

In [ ]:
""" 
Takeaway:
-The ID_Key feature, as is, opperates as an advanced filter to lock in ORI, incident number, 
    and incident date
There is some utility to this, but it does not discriminate unique events as previously desired
Including something like Address, or primary location type, would likely achieve this, however, 
Property Loss file does not have the address feauture, so it would not be able to read such a key.

UNLESS, we knew an offense type/code that would gauruntee a crime also be added to the property loss
file. In that case we could identify those crimes that do actually appear on both. We could match from property
loss to offense file and create a unique code based on if the crime type/code matched a property loss
equivalent. 
"""